# NOTES

In this file, we are showcasing the `v50 (km/s)` computation for every spectrum only for the frequently appeared absorption lines given in the dictionary `frequent_lines`.

# Imports

Loading the libraries, files, and spectrums so we can run preliminary anaylsis.

In [1]:
import pandas as pd
import numpy as np
import os

import astropy.units as u

from scipy.optimize import curve_fit
from specutils import Spectrum1D, SpectralRegion
from astropy.io import fits 

from specutils.manipulation import noise_region_uncertainty, extract_region

# ===========================
#          CONSTANTS
# ===========================
u_wavelength = u.Angstrom 
u_flux       = u.erg / (u.s ** 2) / (u.cm ** 2) / u.Angstrom
u_velocity   = u.km / u.second

light_speed  = 299792.458 * u_velocity

frequent_lines = {
    'C II λ1334': 1334, # 8 times
    'O III λ1666': 1665.85, # 20 times
    'C II λ2326': 2326.0, # 16 times
    'Fe II λ2344': 2344, # 30 times
    'Fe II λ2374': 2374, # 35 times
    'Ne IV λ2440': 2439.5, # 18 times
    'Fe II λ2586': 2586, # 25 times
    'Fe II λ2600': 2600, # 26 times
    'Mg II λ2799': 2799.117, # 26 times
}

In [2]:
hdf5_path = "UltraDeep_Population_Profile.h5"
with pd.HDFStore(hdf5_path, mode='r') as store:
    profile_table = store.get("PopulationProfile")  # Load the DataFrame from the HDF5 file
    print(f"""===== Loaded Table from HDF5 =====
>>> The table has been successfully loaded from '{hdf5_path}' with the key 'PopulationProfile'.""") 
    
    absorption_table = store.get("AbsorptionLineDetection")
    print(f"""===== Loaded Absorption Line Detection Table from HDF5 =====
>>> The table has been successfully loaded from '{hdf5_path}' with the key 'AbsorptionLineDetection'.""")

redshift = profile_table['Z'].values
fits_num = profile_table['NUM'].values
zflag = profile_table['ZFLAGS'].values
stellar_mass = profile_table["STELLAR_MASS"].values # This is in log scale
sfr = profile_table["SFR"].values

redshift = redshift[zflag == 4]
fits_num = fits_num[zflag == 4]
stellar_mass = stellar_mass[zflag == 4]
sfr = sfr[zflag == 4]

===== Loaded Table from HDF5 =====
>>> The table has been successfully loaded from 'UltraDeep_Population_Profile.h5' with the key 'PopulationProfile'.
===== Loaded Absorption Line Detection Table from HDF5 =====
>>> The table has been successfully loaded from 'UltraDeep_Population_Profile.h5' with the key 'AbsorptionLineDetection'.


In [11]:
# ----- Known lines dictionary
known_line = {
    'O VI λ1034': 1033.82,
    'Lyα λ1215': 1215.24,
    'N V λ1241': 1240.81,
    'O I λ1306': 1305.53,
    'C II λ1335': 1335.31,
    'Si IV λ1398': 1397.61,
    'Si IV + O IV λ1400': 1399.8,
    'C IV λ1549': 1549.48,
    'He II λ1640': 1640.4,
    'O III λ1666': 1665.85,
    'Al III λ1857': 1857.4,
    'C III λ1909': 1908.734,
    'C II λ2326': 2326.0,
    'Ne IV λ2440': 2439.5,
    'Mg II λ2799': 2799.117,
    'Ne V λ3347': 3346.79,
    'Ne VI λ3427': 3426.85,
    'O II λ3727': 3727.092,
    'O II λ3730': 3729.875,
    'He I λ3889': 3889.0,
    'K λ3935': 3934.777,
    'H λ3970': 3969.588,
    'Hδ λ4103': 4102.89,
    'G λ4306': 4305.61,
    'Hγ λ4342': 4341.68,
    'O III λ4364': 4364.436,
    'Hβ λ4863': 4862.68,
    'O III λ4933': 4932.603,
    'O III λ4960': 4960.295,
    'O III λ5008': 5008.240,
    'Mg λ5177': 5176.7,
    'Na λ5896': 5895.6,
    'O I λ6302': 6302.046,
    'O I λ6366': 6365.536,
    'N I λ6529': 6529.03,
    'N II λ6550': 6549.86,
    'Hα λ6565': 6564.61,
    'N II λ6585': 6585.27,
    'S II λ6718': 6718.29,
    'S II λ6733': 6732.67,
    'CaII λ8500': 8500.36,
    'CaII λ8544': 8544.44,
    'CaII λ8665': 8664.52,
    'Fe II λ2586': 2586,
    'Fe II λ2600': 2600,
    'Fe II λ2344': 2344,
    'Fe II λ2374': 2374
}

# Functions

## Continuum crossings

In [ ]:
# Function to find first zero crossing on each side of systemic velocity
def find_zero_crossings(velocity, flux):
    """
    Find the first zero crossings (where flux crosses 1.0) on each side of systemic velocity (v=0).
    Doesn't care if it's an absorption or emission feature, just the crossing points.
    
    Parameters:
    -----------
    velocity : np.ndarray
        Velocity array in km/s
    flux : np.ndarray
        Normalized flux array (continuum is at 1.0)
    
    Returns:
    --------
    left_crossing : float or None
        Velocity of first zero crossing on the left side (negative velocity)
    right_crossing : float or None
        Velocity of first zero crossing on the right side (positive velocity)
    """
    # Subtract 1 so that zero crossings are at flux=0
    flux_shifted = flux - 1.0
    
    # Find indices where velocity < 0 (left side) and > 0 (right side)
    left_mask = velocity < 0
    right_mask = velocity > 0
    
    left_crossing = None
    right_crossing = None
    
    # Find first zero crossing on the LEFT side (moving from negative velocity to zero velocity)
    if np.any(left_mask):
        left_vel = velocity[left_mask]
        left_flux = flux_shifted[left_mask]
        
        # Sort by velocity (most negative to zero)
        sort_idx = np.argsort(left_vel)
        left_vel_sorted = left_vel[sort_idx]
        left_flux_sorted = left_flux[sort_idx]
        
        # Find sign changes (zero crossings)
        sign_changes = np.where(np.diff(np.sign(left_flux_sorted)))[0]
        
        if len(sign_changes) > 0:
            # Get the LAST sign change on the left (closest to v=0)
            idx = sign_changes[-1]
            # Linear interpolation to find exact crossing point
            v1, v2 = left_vel_sorted[idx], left_vel_sorted[idx + 1]
            f1, f2 = left_flux_sorted[idx], left_flux_sorted[idx + 1]
            if f2 != f1:
                left_crossing = v1 + (0 - f1) * (v2 - v1) / (f2 - f1)
    
    # Find first zero crossing on the RIGHT side (moving from zero velocity to positive velocities)
    if np.any(right_mask):
        right_vel = velocity[right_mask]
        right_flux = flux_shifted[right_mask]
        
        # Sort by velocity (zero to most positive)
        sort_idx = np.argsort(right_vel)
        right_vel_sorted = right_vel[sort_idx]
        right_flux_sorted = right_flux[sort_idx]
        
        # Find sign changes (zero crossings)
        sign_changes = np.where(np.diff(np.sign(right_flux_sorted)))[0]
        
        if len(sign_changes) > 0:
            # Get the FIRST sign change on the right (closest to v=0)
            idx = sign_changes[0]
            # Linear interpolation to find exact crossing point
            v1, v2 = right_vel_sorted[idx], right_vel_sorted[idx + 1]
            f1, f2 = right_flux_sorted[idx], right_flux_sorted[idx + 1]
            if f2 != f1:
                right_crossing = v1 + (0 - f1) * (v2 - v1) / (f2 - f1)
    
    return left_crossing, right_crossing

## Absorption area and V50

In [5]:
# Function to compute absorption area within wings
def compute_absorption_area(velocity, flux, left_crossing, right_crossing):
    """
    Compute the area between continuum (flux=1) and normalized flux within the absorption wings
    using trapezoidal integration.
    
    Parameters:
    -----------
    velocity : np.ndarray
        Velocity array in km/s
    flux : np.ndarray
        Normalized flux array (continuum is at 1.0)
    left_crossing : float or None
        Left wing boundary velocity
    right_crossing : float or None
        Right wing boundary velocity
    
    Returns:
    --------
    area : float or None
        Integrated area between continuum and flux (absorption strength)
        Units: km/s (velocity integrated flux deficit)
    """
    if left_crossing is None or right_crossing is None:
        return None
    
    # Extract data within the wings
    mask = (velocity >= left_crossing) & (velocity <= right_crossing)
    vel_region = velocity[mask]
    flux_region = flux[mask]
    
    if len(vel_region) < 2:
        return None
    
    # Calculate area: integral of (1 - flux) dv
    # This is the flux deficit below continuum
    flux_deficit = 1.0 - flux_region
    
    # Use trapezoidal integration
    area = np.trapz(flux_deficit, vel_region)
    
    return area

In [6]:
# Function to compute v50 (median velocity)
def compute_v50(velocity, flux, left_crossing, right_crossing):
    """
    Compute the v50 velocity - the velocity at which half of the absorption area 
    has been accumulated when integrating from the left wing.
    
    Parameters:
    -----------
    velocity : np.ndarray
        Velocity array in km/s
    flux : np.ndarray
        Normalized flux array (continuum is at 1.0)
    left_crossing : float or None
        Left wing boundary velocity
    right_crossing : float or None
        Right wing boundary velocity
    
    Returns:
    --------
    v50 : float or None
        Velocity at which half the absorption area is accumulated
        Units: km/s
    """
    if left_crossing is None or right_crossing is None:
        return None
    
    # Extract data within the wings
    mask = (velocity >= left_crossing) & (velocity <= right_crossing)
    vel_region = velocity[mask]
    flux_region = flux[mask]
    
    if len(vel_region) < 2:
        return None
    
    # Calculate flux deficit
    flux_deficit = 1.0 - flux_region
    
    # Compute cumulative area from left to right
    cumulative_area = np.zeros(len(vel_region))
    for i in range(1, len(vel_region)):
        # Trapezoidal rule for each segment
        dv = vel_region[i] - vel_region[i-1]
        avg_deficit = (flux_deficit[i] + flux_deficit[i-1]) / 2.0
        cumulative_area[i] = cumulative_area[i-1] + avg_deficit * dv
    
    # Total area
    total_area = cumulative_area[-1]
    
    if total_area <= 0:
        return None
    
    # Find where cumulative area reaches half
    half_area = total_area / 2.0
    
    # Find the index where we cross half the area
    idx = np.searchsorted(cumulative_area, half_area)
    
    if idx == 0:
        return vel_region[0]
    if idx >= len(vel_region):
        return vel_region[-1]
    
    # Linear interpolation to get precise v50
    v1, v2 = vel_region[idx-1], vel_region[idx]
    a1, a2 = cumulative_area[idx-1], cumulative_area[idx]
    
    if a2 != a1:
        v50 = v1 + (half_area - a1) * (v2 - v1) / (a2 - a1)
    else:
        v50 = v1
    
    return v50

## Miscellaneous

In [9]:
# ----- Get systemic velocity from redshift 
def vsys(z:float):
    return light_speed * ((z + 1) ** 2 - 1) / ((z + 1) ** 2 + 1)

# ----- Chech if a value is in spectral region
def is_inregion(val, region:SpectralRegion):
    """Check if a value is inside a (singular) spectral region.

    Args:
        val (float * unit): A value to check
        region (SpectralRegion): The spectral region of interest. The spectral region 
                                 must be singular in a sense that it can be written as (a, b).

    Raises:
        Exception: If unit of val and region isn't the same.

    Returns:
        bool: Whether or not the value of interest is inside the region.
    """
    if val.unit != region.lower.unit:
        raise Exception(f"Unit Error: The units of inserted value is {val.unit} but SpectralRegion is in units {region.lower.unit}.")
    
    if not isinstance(val, float):
        val = val.value 
    return region.lower.value <= val and val <= region.upper.value

## Quiet regions

In [10]:
# ----- Get quietregion in observed frame (around known lines)
def get_quietregion(redshift:float, scale_threshold:float = 3) -> SpectralRegion:
    """Extract regions that might not have major emission line. The extracted region is a subset of 
    3500 A and 9467.18 A. This is done by deleting the region around known_lines.

    Args:
        redshift (float): the redshift of a galaxy
        scale_threshold (float, optional): Sampling step of the wavelength in observed frame is 5.355 A. 
                                           So, we use this parameter to extend/shrink the deleted region. Defaults to 3.

    Returns:
        SpectralRegion: Spectral region of no major emission line regions.
    """
    known_lam = np.array([*known_line.values()])
    
    # Sampling step = 5.355
    known_lamobs = known_lam * (1 + redshift) # Known_lam in observed frame. 
    quiet_region = [(known_lamobs[i], known_lamobs[i+1]) for i in range(len(known_lam) - 1) if known_lamobs[i+1] - known_lamobs[i] >= 100]
    
    if quiet_region[0][1] > 9467.18: # Too high redshift, no quiet region from the available known line.
        return None 
    
    # lam_threshold is the distance between known line's center and the edge which we want to exclude from the quiet region
    # we do this to avoid potential remnants of emission line or absorption line
    lam_threshold = scale_threshold * 5.355
    
    return_region = SpectralRegion(0*u.nm, 1*u.nm) # Template and will be removed later
    for index in range(len(quiet_region)):
        a = quiet_region[index][0]
        b = quiet_region[index][1]
        
        # We want only subregions bounded by 3500 AA and 9467.18 AA because that's the observable frame's boundary
        if 3500 <= a and b <= 9467.18: 
            return_region += SpectralRegion((a + lam_threshold) * u.Angstrom, (b - lam_threshold) * u.Angstrom)
        
    del return_region[0]
    return return_region

# ----- Get the arithmetic average of the 1sigma noise within quiet region.
def get_quietregion_sigma(lam:np.ndarray, noise:np.ndarray, redshift:float, scale_threshold:float = 3) -> float:
    """Calculate a representative standard deviation of the noise of the quiet region. This representative
    sigma is done by averaging the (absolute value of) noise in quiet region.

    Args:
        lam (np.ndarray): _description_
        noise (np.ndarray): _description_
        redshift (float): _description_
        scale_threshold (float, optional): _description_. Defaults to 3.

    Raises:
        ValueError: _description_

    Returns:
        float: _description_
    """
    if lam is None or noise is None:
        raise ValueError("Need proper inputs!")
    
    quietregion = get_quietregion(redshift, scale_threshold)
    mask = get_quietregion_mask(lam, quietregion)
    
    if sum(mask) == 0:
        return None
    
    return noise[mask].sum() / len(noise[mask])

# ----- Get mask for points in quietregion
def get_quietregion_mask(lam:np.ndarray, quietregion:SpectralRegion) -> np.ndarray[bool]:
    return np.array([is_inregion(wavelength * u.Angstrom, quietregion) for wavelength in lam])

## Spectrum Processing

In [12]:
# ----- Binning data
def get_bindata(data, bins, binsmode="med"):
    """Obtain a binned array by various method of binning.

    Args:
        data (np.ndarray): Numpy array of data of interest.
        bins (int): Number of data points that we want 1 point in the binned data to represent. Like, if bins = 3
                    then we use 1 point in the binned data to represent 3 points in the original data. 
        binsmode (str, optional): _description_. Defaults to "med".

    Raises:
        ValueError: Invalid mode of binning.

    Returns:
        np.ndarray: Numpy array of binned data.
    """
    binsmode = binsmode.lower()
    match binsmode:
        case "med":
            binned_data = np.array([np.median(data[i:i+bins]) for i in range(len(data) - (bins - 1))])
        case "mean":
            binned_data = np.array([np.mean(data[i:i+bins]) for i in range(len(data) - (bins - 1))])
        case "sum":
            binned_data = np.array([np.sum(data[i:i+bins]) for i in range(len(data) - (bins - 1))])
        case _:
            raise ValueError("Invalid binsmode. Choose from 'med', 'mean', or 'sum'.")
    return binned_data

# ----- Get atm_clean fits file of an observation field
def get_cleanfitsfile(fieldname:str, file_index:int):
    """Obtain a .fits file of a spectrum from the downloaded .fits file of a certain observation field.

    Args:
        fieldname (str): Name of the observation field of interest. ("ud", "df02", "dcdfs")
        file_index (int): Index of a specific field in the field's directory.

    Returns:
        _type_: Spectrum1D of the spectrum and noise.
    """
    match fieldname:
        case "ud":
            spec_dir = "..\\Flagged Data\\VIMOS VLT Ultra Deep\\atm_clean\\" 
            noise_dir = "..\\Flagged Data\\VIMOS VLT Ultra Deep\\noise\\" 
            spec_files = os.listdir(spec_dir)
            noise_files = os.listdir(noise_dir)
        case "df02":
            spec_dir = "..\\Flagged Data\\VIMOS VLT Deep\\VVDS-F0226-04\\atm_clean\\" 
            noise_dir = "..\\Flagged Data\\VIMOS VLT Deep\\VVDS-F0226-04\\noise\\" 
            spec_files = os.listdir(spec_dir)
            noise_files = os.listdir(noise_dir)
        case "dcdfs":
            spec_dir = "..\\Flagged Data\\VIMOS VLT Deep\\VVDS-CDFS\\atm_clean\\" 
            noise_dir = "..\\Flagged Data\\VIMOS VLT Deep\\VVDS-CDFS\\noise\\" 
            spec_files = os.listdir(spec_dir)
            noise_files = os.listdir(noise_dir)

    spec_file = fits.open(spec_dir + spec_files[file_index], memmap = True)
    noise_file = fits.open(noise_dir + noise_files[file_index], memmap = True)
    
    return spec_file, noise_file

# ----- Get Spectrum1D object from fits file.
def get_spec1d(fieldname, file_index, redshift, bins = 1, binsmode = "med", fits_num = None, quiet_fluxsigma = True):
    """Obtain Spectrum1D object of a spectrum from the .fits file of a certain observation field.

    Args:
        fieldname (str): Name of the observation field of interest.
        file_index (int): Index of spectrum .fits file within the field's directory.
        redshift (float): redshift of the galaxy of interest.
        bins (int, opetional): number of data points taht we want 1 point in the binned data
                               to represent. Defaults is 1. (no binning)
        binsmode (str, optional): binning mode of the flux array. Defaults to median.
        fits_num (int, optional): .fits ID in the VVDS database. Defaults to None.
        quiet_fluxsigma (bool, optional): Whether or not to use the standard deviation of flux to represent
                                          the uncertainty in quiet region. Defaults to True.

    Returns:
        Specturm1D: Spectrum1D object of the spectrum of interest.
    """
    
    if quiet_fluxsigma: 
        spec_fits, _ = get_cleanfitsfile(fieldname, file_index)
    else:
        spec_fits, noise_fits = get_cleanfitsfile(fieldname, file_index)
        noise = noise_fits[0].data[0]
    
    frame = "obs" # Spectrum in fits file is in observed frame.
    
    spec_header = spec_fits[0].header
    flux = spec_fits[0].data[0]
    lam = spec_header['CRVAL1'] + spec_header['CDELT1'] * np.arange(len(flux))
    
    spec_meta = {"fitsnum": fits_num,
                 "frame": frame, 
                 "redshift": redshift}

    if bins > 1:
        lam = get_bindata(lam, bins, binsmode = "mean")
        flux = get_bindata(flux, bins, binsmode)
        if not quiet_fluxsigma:
            noise = get_bindata(noise, bins, binsmode)
            spec_meta.update({"noise": noise * u_flux})
    
    if quiet_fluxsigma:
        spec_obj = Spectrum1D(spectral_axis = lam * u_wavelength, flux = flux * u_flux)
        quietsigma = noise_region_uncertainty(spec_obj, get_quietregion(redshift))
        quietsigma = quietsigma.uncertainty.array[0]
    else: 
        quietsigma = get_quietregion_sigma(lam, flux, redshift)
        
    spec_meta.update({"quietsigma": quietsigma * u_flux})
    
    spec = Spectrum1D(spectral_axis = lam * u_wavelength,
                      flux = flux * u_flux,
                      meta = spec_meta)
    
    return spec 

# ----- Get matching known lines
def get_linematching(detected_lines, redshift, frame = "obs", line_type = "absorption", tolerance = 50):
    """Obtain a dictionary of matching known lines from detected lines from find_lines_threshold. The key of the output
    dictionary is the candidate known line's name while the value is the detected line center wavelength (in Angstrom).

    Args:
        detected_lines (QTable): detected lines from find_lines_threshold
        redshift (float): redshift of the spectrum of interest
        frame (str, optional): frame of the output matching lines. Choose from "obsserved" or "rest". Defautls to "obs".
        line_type (str, optional): Line type detected by find_lines_threshold that we want to match. Defaults to "absorption".
        tolerance (int, optional): Maximum line deviation from known_line wavelength (in Angstrom). Defaults to 10.

    Raises:
        ValueError: Invalid frame.

    Returns:
        dict: Matching known lines dictionary. Key: known line name, Value: deteceted line center wavelength (in Angstrom)
    """
    # ----- Helper function -----
    # Get the NEAREST match only from the matching list (there might be multiple detected_lam matching to 1 known_lam)
    def get_nearest_match(matching, known_center): 
        candidates = [match for match in matching if match[1] == known_center]
        if len(candidates) == 0:
            return None 
        candidates.sort(key = lambda x: x[2]) # Sort by np.abs(detected_lam - known_center)
        return candidates[0]
    
    if line_type in ("absorption", "emission"):
        detected_lines = detected_lines[detected_lines['line_type'] == line_type]
    
    known_name = list(known_line.keys())
    known_lam = np.array([*known_line.values()]) * (1 + redshift) # In observed frame 
    detected_lam = detected_lines['line_center'].to(u.Angstrom).value 
    matching = []
    
    for known_center in known_lam:
        idx = np.abs(detected_lam - known_center).argmin()
        if np.abs(detected_lam[idx] - known_center) < tolerance:
            matching.append((detected_lam[idx], known_center, np.abs(detected_lam[idx] - known_center)))

    final_match = [get_nearest_match(matching, known_center) for known_center in known_lam]
    final_match = [match for match in final_match if match is not None]

    match_dict = {}

    for match in final_match:
        detected_center, known_center, _ = match
        known_idx = known_lam.tolist().index(known_center)
        known_name_iter = known_name[known_idx]
        match_dict[known_name_iter] = detected_center
    
    if frame[0] == "o":
        return match_dict 
    elif frame[0] == "r":
        # Convert to rest frame 
        match_dict = {name: lam / (1 + redshift) for name, lam in match_dict.items()}
        return match_dict
    else:
        raise ValueError("Invalid frame. Choose from \"obs\" or \"rest\".")

# Creating the Table containing v50's.

In this section, we will create a `pandas.DataFrame` containing the `v50` of each spectral lines in the `frequent_lines` dictionary. If we can't find the `v50` of that line, we will let `v50 = None` (perhaps at that line it's an emission or the line isn't a part of the observed spectrum).

In [34]:
v50_table = pd.DataFrame(columns=["NUM"] + list(frequent_lines.keys()))

## Getting v50's from every spectrums

In [35]:
def poly_fit(x, a0, a1, a2, a3, a4):
    return a0 + a1*x + a2*x**2 + a3*x**3 + a4*x**4

In [36]:
for file_index in range(len(redshift)):
    # ----- Continuum normalization -----
    z = redshift[file_index]
    fits_id = fits_num[file_index]
    spec = get_spec1d("ud", file_index, z, fits_num = fits_id)
    
    popt, _ = curve_fit(poly_fit, spec.spectral_axis.value, spec.flux.value)
    con_flux = poly_fit(spec.spectral_axis.value, *popt)
    
    connorm_flux = spec.flux.value / con_flux
    spec_norm = Spectrum1D(spectral_axis = spec.spectral_axis, flux = connorm_flux * u_flux)
    
    # ----- v50 calculation for frequent absorption lines -----
    absorption_results = {}

    for idx, (line_name, line_center) in enumerate(frequent_lines.items()):
        line_center_obs = line_center * (1 + z) * u.Angstrom
        vel_axis = light_speed * (spec.spectral_axis - line_center_obs) / line_center_obs
        
        # Extract region around the line center
        region = SpectralRegion(line_center_obs - 500 * u.Angstrom, line_center_obs + 500 * u.Angstrom)
        spec_line = extract_region(spec_norm, region)
        
        vel_line = light_speed * (spec_line.spectral_axis - line_center_obs) / line_center_obs
        flux_line = spec_line.flux.value
        
        left_crossing, right_crossing = find_zero_crossings(vel_line.value, flux_line)
        area = compute_absorption_area(vel_line.value, flux_line, left_crossing, right_crossing)
        v50 = compute_v50(vel_line.value, flux_line, left_crossing, right_crossing)
        
        absorption_results[line_name] = {
            "left_crossing": left_crossing,
            "right_crossing": right_crossing,
            "area": area,
            "v50": v50
        }

    v50_dict = {line_name: absorption_results[line_name]["v50"] for line_name in frequent_lines.keys()}
    v50_tablerow = {"NUM": fits_id, **v50_dict}
    v50_table.loc[len(v50_table)] = v50_tablerow

In [39]:
v50_table

,NUM,C II λ1334,O III λ1666,C II λ2326,Fe II λ2344,Fe II λ2374,Ne IV λ2440,Fe II λ2586,Fe II λ2600,Mg II λ2799
0,910161060,2481.932745,-43.843288,NaN,-413.501362,536.563822,NaN,525.001427,-1092.092585,-511.833830
1,910166908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,910167444,1197.190109,530.574111,-168.393976,-31.857916,NaN,NaN,NaN,NaN,NaN
3,910170531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,910170571,NaN,NaN,NaN,NaN,426.719069,NaN,260.665260,55.154028,NaN
...,...,...,...,...,...,...,...,...,...,...
144,910376825,-605.609746,437.757986,NaN,-479.409238,39.593217,-504.763695,-926.180619,55.150986,NaN
145,910378108,NaN,NaN,NaN,600.886488,576.439525,-276.508260,NaN,-45.055786,-663.492378
146,910378821,NaN,NaN,NaN,NaN,654.755599,NaN,445.857186,425.827269,NaN
147,910379064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2746.280165,NaN


## Transforming to HDF5

In [40]:
with pd.HDFStore(hdf5_path, mode='a') as store:
    store.put("Absorption_V50_Table", v50_table, format='table')
    print(f"""===== Saved v50 Table to HDF5 =====
>>> The v50 table has been successfully saved to '{hdf5_path}' with the key 'Absorption_V50_Table'.""")

===== Saved v50 Table to HDF5 =====
>>> The v50 table has been successfully saved to 'UltraDeep_Population_Profile.h5' with the key 'Absorption_V50_Table'.
